In [1]:
import time
import pandas as pd
import numpy as np
import optuna
from pycaret.classification import *
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
import sys 
print(sys.version)

3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]


# READ ALL THE DATA

In [3]:
train = pd.read_csv("data/train.csv")
train.head()

,individualnumber,category_number,hakkedis_amt,odul_amt,response
0,94230288,9000,21.0,2.0,0
1,4684087,9000,17.0,1.0,0
2,92472145,9058,24.0,3.0,0
3,88026681,9030,22.0,2.0,0
4,98127795,9001,38.0,3.0,0


In [4]:
test = pd.read_csv("data/test.csv")
test.head()

,individualnumber,category_number,hakkedis_amt,odul_amt
0,95812936,9029,38.0,3.0
1,23929570,9012,28.0,2.0
2,95948115,9019,21.0,2.0
3,100898513,9035,48.0,4.0
4,12769156,9015,26.0,3.0


In [5]:
customer = pd.read_csv("data/customer.csv")
customer.head()

,individualnumber,gender,city_code,dateofbirth
0,94212124,K,NaN,1946.0
1,96387515,E,34.0,1974.0
2,95040383,E,80.0,1980.0
3,94694434,E,NaN,1970.0
4,47648671,E,35.0,1963.0


In [6]:
customeraccount = pd.read_csv("data/customeraccount.csv")
customeraccount.head()

,individualnumber,cardnumber
0,16481068,7287134635560315
1,17634947,6587004647560415
2,19323290,8187034648564315
3,19323290,3005502487247749
4,37820213,6287114610560316


In [7]:
genel_kategoriler = pd.read_csv("data/genel_kategoriler.csv")
genel_kategoriler.head()

,category_number,genel_kategori
0,9052,diger
1,9053,diger
2,9054,diger
3,9055,gida
4,9056,gida


In [8]:
product_groups = pd.read_csv("data/product_groups.csv")
product_groups.head()

,category_number,category_level_1,category_level_2,category_level_3,category_level_4
0,9046,341,20,10,10
1,9009,125,25,10,10
2,9049,410,65,20,10
3,9019,300,10,19,11
4,9035,305,31,10,10


In [9]:
transaction_header = pd.read_csv("data/transaction_header.csv")
transaction_header['date_of_transaction'] = pd.to_datetime(transaction_header['date_of_transaction']).dt.strftime('%d/%m/%Y')
transaction_header = transaction_header[transaction_header['date_of_transaction'].notna()]
transaction_header.head()

,date_of_transaction,cardnumber,basketid,is_sanal
0,01/12/2020,8487374630565615,20120163990041163,0
1,01/12/2020,4287574603569015,20120163990016137,0
2,01/12/2020,6487154648560315,20120160080017338,0
3,01/12/2020,3787564693563915,20120163570105022,0
4,01/12/2020,9087994683569715,20120162340016645,0


In [10]:
transaction_sale = pd.read_csv("data/transaction_sale.csv")
transaction_sale.rename(columns = {'category_level_1':'ts_category_level_1', 'category_level_2':'ts_category_level_2',
                                  'category_level_3':'ts_category_level_3', 'category_level_4':'ts_category_level_4',}, 
                        inplace = True)
transaction_sale.head()

,basketid,ts_category_level_1,ts_category_level_2,ts_category_level_3,ts_category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,20120706070177471,613,50,50,15,19.9,2.0,NaN,0.0,0.0
1,20120706070177471,425,30,12,10,3.5,1.0,NaN,0.0,0.0
2,20120706070177471,425,30,12,11,7.9,1.0,NaN,0.0,0.0
3,20122703940031323,103,25,15,10,7.9,2.0,NaN,0.0,0.0
4,20122703940031323,101,15,15,15,16.0,1.0,NaN,0.0,0.0


# OPTIMIZE MEMORY USAGE FOR DATAFRAMES

In [ ]:
print(train.info(memory_usage = "deep"), end = "\n\n")
print(train.describe())

In [ ]:
train["category_number"] = train.category_number.astype(np.int16)
train["hakkedis_amt"] = train.hakkedis_amt.astype(np.int8)
train["odul_amt"] = train.odul_amt.astype(np.int8)
train["response"] = train.response.astype(bool)

In [ ]:
train.info(memory_usage = "deep")

In [ ]:
print(test.info(memory_usage = "deep"), end = "\n\n")
print(test.describe())

In [ ]:
test["category_number"] = test.category_number.astype(np.int16)
test["hakkedis_amt"] = test.hakkedis_amt.astype(np.int8)
test["odul_amt"] = test.odul_amt.astype(np.int8)

In [ ]:
test.info(memory_usage = "deep")

In [ ]:
print(customer.info(memory_usage = "deep"), end = "\n\n")
print(customer.describe())

In [ ]:
customer["gender"].fillna("Unknown", inplace=True)
customer["city_code"].fillna(-1, inplace=True)
customer["dateofbirth"].fillna(-1, inplace=True)

customer["gender"] = customer.gender.astype("category")
customer["city_code"] = customer.city_code.astype(np.int8)
customer["dateofbirth"] = customer.dateofbirth.astype(np.int16)

In [ ]:
customer.info(memory_usage = "deep")

In [ ]:
print(customeraccount.info(memory_usage = "deep"), end = "\n\n")
print(customeraccount.describe())

In [ ]:
print(genel_kategoriler.info(memory_usage = "deep"), end = "\n\n")
print(genel_kategoriler.describe())

In [ ]:
genel_kategoriler["category_number"] = genel_kategoriler.category_number.astype(np.int16)
genel_kategoriler["genel_kategori"] = genel_kategoriler.genel_kategori.astype("category")

In [ ]:
print(genel_kategoriler.info(memory_usage = "deep"))

In [ ]:
print(product_groups.info(memory_usage = "deep"), end = "\n\n")
print(product_groups.describe())

In [ ]:
product_groups["category_number"] = product_groups.category_number.astype(np.int16)
product_groups["category_level_1"] = product_groups.category_level_1.astype(np.int16)
product_groups["category_level_2"] = product_groups.category_level_2.astype(np.int8)
product_groups["category_level_3"] = product_groups.category_level_3.astype(np.int8)
product_groups["category_level_4"] = product_groups.category_level_4.astype(np.int8)

In [ ]:
print(product_groups.info(memory_usage = "deep"))

In [ ]:
print(transaction_header.info(memory_usage = "deep"), end = "\n\n")
print(transaction_header.describe())

In [ ]:
transaction_header["date_of_transaction"] = transaction_header.date_of_transaction.astype("category")
transaction_header["is_sanal"] = transaction_header.is_sanal.astype(np.int8)

In [ ]:
print(transaction_header.info(memory_usage = "deep"))

In [ ]:
print(transaction_sale.info(memory_usage = "deep"), end = "\n\n")
print(transaction_sale.describe())

In [ ]:
transaction_sale["ts_category_level_1"] = transaction_sale.ts_category_level_1.astype(np.int16)
transaction_sale["ts_category_level_2"] = transaction_sale.ts_category_level_2.astype(np.int16)
transaction_sale["ts_category_level_3"] = transaction_sale.ts_category_level_3.astype(np.int16)
transaction_sale["ts_category_level_4"] = transaction_sale.ts_category_level_4.astype(np.int16)
transaction_sale["amount"] = transaction_sale.amount.astype(np.float16)
transaction_sale["quantity"] = transaction_sale.quantity.astype(np.float16)
transaction_sale["discount_type_1"] = transaction_sale.discount_type_1.astype(np.float16)
transaction_sale["discount_type_2"] = transaction_sale.discount_type_2.astype(np.float16)
transaction_sale["discount_type_3"] = transaction_sale.discount_type_3.astype(np.float16)

In [ ]:
print(transaction_sale.info(memory_usage = "deep"))

# MERGE TABLES

In [11]:
%%notify

train_customer = pd.merge(train.drop_duplicates(subset=['individualnumber']), customer, how="left")
train_customer_customeraccount = pd.merge(train_customer.drop_duplicates(subset=['individualnumber']), customeraccount, how="left")
train_customer_customeraccount_genelkategoriler =  pd.merge(train_customer_customeraccount.drop_duplicates(subset=['individualnumber']), genel_kategoriler, how="left")
train_customer_customeraccount_genelkategoriler_productgroups =  pd.merge(train_customer_customeraccount_genelkategoriler.drop_duplicates(subset=['individualnumber']), product_groups, how="left")
train_customer_customeraccount_genelkategoriler_productgroups_transactionheader =  pd.merge(train_customer_customeraccount_genelkategoriler_productgroups.drop_duplicates(subset=['individualnumber']), transaction_header, how="left")
train_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale =  pd.merge(train_customer_customeraccount_genelkategoriler_productgroups_transactionheader.drop_duplicates(subset=['individualnumber']), transaction_sale, how="left")

train_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.drop("date_of_transaction", axis=1, inplace=True)
train_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale = train_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.drop_duplicates(subset=['individualnumber'])

train_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.head()

,individualnumber,category_number,hakkedis_amt,odul_amt,response,gender,city_code,dateofbirth,cardnumber,genel_kategori,category_level_1,category_level_2,category_level_3,category_level_4,basketid,is_sanal,ts_category_level_1,ts_category_level_2,ts_category_level_3,ts_category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,94230288,9000,21.0,2.0,0,E,7.0,1983.0,787714605562415,icecek,603,25,10,10,2.012076e+16,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4684087,9000,17.0,1.0,0,E,19.0,1951.0,9087854623560519,icecek,603,25,10,10,2.101044e+16,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,92472145,9058,24.0,3.0,0,K,35.0,1982.0,4087494610563715,gida,617,55,10,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,88026681,9030,22.0,2.0,0,K,NaN,1961.0,7087024687560515,kisisel_bakim,628,25,10,15,2.012124e+16,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,98127795,9001,38.0,3.0,0,E,9.0,1984.0,9687474690567815,gida,110,10,10,10,2.103226e+16,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [12]:
%%notify

test_customer = pd.merge(test.drop_duplicates(subset=['individualnumber']), customer, how="left")
test_customer_customeraccount = pd.merge(test_customer.drop_duplicates(subset=['individualnumber']), customeraccount, how="left")
test_customer_customeraccount_genelkategoriler =  pd.merge(test_customer_customeraccount.drop_duplicates(subset=['individualnumber']), genel_kategoriler, how="left")
test_customer_customeraccount_genelkategoriler_productgroups =  pd.merge(test_customer_customeraccount_genelkategoriler.drop_duplicates(subset=['individualnumber']), product_groups, how="left")
test_customer_customeraccount_genelkategoriler_productgroups_transactionheader =  pd.merge(test_customer_customeraccount_genelkategoriler_productgroups.drop_duplicates(subset=['individualnumber']), transaction_header, how="left")
test_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale =  pd.merge(test_customer_customeraccount_genelkategoriler_productgroups_transactionheader.drop_duplicates(subset=['individualnumber']), transaction_sale, how="left")

test_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.drop("date_of_transaction", axis=1, inplace=True)
test_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale = test_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.drop_duplicates(subset=['individualnumber'])

test_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.head()

,individualnumber,category_number,hakkedis_amt,odul_amt,gender,city_code,dateofbirth,cardnumber,genel_kategori,category_level_1,category_level_2,category_level_3,category_level_4,basketid,is_sanal,ts_category_level_1,ts_category_level_2,ts_category_level_3,ts_category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,95812936,9029,38.0,3.0,E,10.0,1995.0,987444691569515,kisisel_bakim,628,15,10,35,2.105175e+16,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23929570,9012,28.0,2.0,E,34.0,1974.0,5687824656560416,gida,617,65,10,10,2.012203e+16,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,95948115,9019,21.0,2.0,E,67.0,1967.0,5687474662560115,gida,300,10,19,11,2.101226e+16,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100898513,9035,48.0,4.0,K,51.0,1978.0,8187204608568215,gida,305,31,10,10,2.104027e+16,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12769156,9015,26.0,3.0,K,34.0,1970.0,4787834626560515,gida,616,80,15,15,2.012011e+16,0.0,311.0,15.0,10.0,20.0,13.53,0.68,NaN,2.04,0.0


<IPython.core.display.Javascript object>

In [13]:
train_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.shape

(13115, 25)

In [14]:
test_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale.shape

(13236, 24)

In [15]:
train_all_merged = train_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale
test_all_merged = test_customer_customeraccount_genelkategoriler_productgroups_transactionheader_transactionsale

In [16]:
train_data = train_all_merged
test_data = test_all_merged

In [73]:
%%notify

s = setup(data = train_data, 
          target = 'response', 
          experiment_name = 'migros',
          data_split_stratify = True,
          fold = 10,
          use_gpu = True,
          normalize = True,
          pca = False,
          remove_outliers = True,
          remove_multicollinearity = True,
          feature_selection = True,
          #feature_selection_method = "boruta",
          fix_imbalance = True,
          silent = True,
          )

,Description,Value
0,session_id,2774
1,Target,response
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(13115, 25)"
5,Missing Values,True
6,Numeric Features,18
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


<IPython.core.display.Javascript object>

In [72]:
%%notify

best_models = compare_models(sort="F1", include = ["gbc", "ada", "lightgbm"], n_select = 5, fold = 10, turbo = True)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9417,0.8401,0.2643,0.0844,0.1273,0.1056,0.1246,3.4990
ada,Ada Boost Classifier,0.9025,0.8180,0.4429,0.0734,0.1258,0.1011,0.1496,0.8930
lightgbm,Light Gradient Boosting Machine,0.9775,0.8208,0.0429,0.0942,0.0583,0.0485,0.0527,2.6950


<IPython.core.display.Javascript object>

In [74]:
%%notify

best_model = best_models[2]
#best_model = create_model("rf")

<IPython.core.display.Javascript object>

In [75]:
%%notify

tuned_model = tune_model(best_model, optimize = "F1", n_iter = 50, search_library = 'optuna', choose_better = True, 
                         early_stopping=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9599,0.8657,0.2143,0.1111,0.1463,0.1279,0.1352
1,0.9610,0.7925,0.1538,0.0800,0.1053,0.0874,0.0923
2,0.9472,0.8159,0.1538,0.0541,0.0800,0.0592,0.0680
3,0.9610,0.7353,0.1538,0.0800,0.1053,0.0874,0.0923
4,0.9644,0.8978,0.5385,0.2188,0.3111,0.2962,0.3283
5,0.9518,0.8288,0.0000,0.0000,0.0000,-0.0219,-0.0233
6,0.9553,0.7551,0.1429,0.0690,0.0930,0.0729,0.0781
7,0.9736,0.8635,0.2857,0.2353,0.2581,0.2448,0.2460
8,0.9656,0.8859,0.2857,0.1667,0.2105,0.1942,0.2016
9,0.9576,0.7796,0.2857,0.1290,0.1778,0.1592,0.1726


<IPython.core.display.Javascript object>

In [76]:
# default model
print(best_model)

# tuned model
print(tuned_model)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=7619, reg_alpha=0.0, reg_lambda=0.0,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
LGBMClassifier(bagging_fraction=0.8641506375381373, bagging_freq=4,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', feature_fraction=0.9757641610215504,
               importance_type='split', learning_rate=0.11406676504183538,
               max_depth=-1, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.2873063997050293, n_estimators=43, n_jobs=-1,
               num_leaves=9, objective=None, random_state=7619,
      

In [77]:
%%notify

final_model = finalize_model(tuned_model)

[LightGBM] [Warning] feature_fraction is set=0.9757641610215504, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9757641610215504
[LightGBM] [Warning] bagging_fraction is set=0.8641506375381373, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8641506375381373
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.9757641610215504, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9757641610215504
[LightGBM] [Warning] bagging_fraction is set=0.8641506375381373, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8641506375381373
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.9757641610215504, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9757641610215504
[LightGBM] [Warning] bagging

<IPython.core.display.Javascript object>

In [82]:
save_model(final_model, "final model")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='response',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_st...
                                 learning_rate=0.11406676504183538, max_depth=-1,
                                 min_child_samples=80, min_child_weight=0.001,
                                 min_split_gain=0.2873063997050293,
                                 n_estimators

In [23]:
%%notify

unprocessed_prediction = predict_model(final_model, data = test_data)
unprocessed_prediction

,individualnumber,category_number,hakkedis_amt,odul_amt,gender,city_code,dateofbirth,cardnumber,genel_kategori,category_level_1,...,ts_category_level_2,ts_category_level_3,ts_category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3,Label,Score
0,95812936,9029,38.0,3.0,E,10.0,1995.0,987444691569515,kisisel_bakim,628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.5271
1,23929570,9012,28.0,2.0,E,34.0,1974.0,5687824656560416,gida,617,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.5010
2,95948115,9019,21.0,2.0,E,67.0,1967.0,5687474662560115,gida,300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.5220
3,100898513,9035,48.0,4.0,K,51.0,1978.0,8187204608568215,gida,305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.5229
4,12769156,9015,26.0,3.0,K,34.0,1970.0,4787834626560515,gida,616,...,15.0,10.0,20.0,13.53,0.68,NaN,2.04,0.0,0,0.5149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31763,93418529,9049,32.0,3.0,E,38.0,1979.0,887284617560415,diger,410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.5128
31764,97397069,9018,20.0,2.0,E,9.0,1993.0,6487034685567815,gida,131,...,10.0,10.0,10.0,0.50,2.00,NaN,0.00,0.0,0,0.5130
31774,95212479,9046,85.0,8.0,E,NaN,1962.0,7287954643563815,gida,341,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.5037
31775,93406449,9032,39.0,3.0,E,78.0,1966.0,9487234673567915,kisisel_bakim,220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.5243


<IPython.core.display.Javascript object>

In [24]:
submission = pd.DataFrame({"individualnumber":test["individualnumber"]})
submission["response"] = list(unprocessed_prediction["Label"].astype(int))
submission = submission.sort_index(ascending=True)
submission

,individualnumber,response
0,95812936,0
1,23929570,1
2,95948115,0
3,100898513,0
4,12769156,0
...,...,...
13231,93418529,0
13232,97397069,0
13233,95212479,0
13234,93406449,0


In [25]:
submission["response"].value_counts()

0    12438
1      798
Name: response, dtype: int64

In [26]:
submission.to_csv(f"submission/Submission-{time.strftime('%d - %H-%M-%S')}.csv", index=False)